# NBA Free Agency

This notebook aims to determine whether the top 10 NBA free agents (or players who opt for player option in their contracts) are likely stay or leave a team during free agency or the mid-season trade deadline. This will be determined using NLP techniques on phrases extracted from tweets, news, interviews, polls, basketball stats, etc. 

#### Top 10 NBA Free Agents 2019
According to SBNation and ESPN these players are: <br>
Link: https://www.sbnation.com/nba/2018/7/30/17616436/nba-free-agency-2019-list-kevin-durant-kyrie-irving

1. Kevin Durant
2. Kawhi Leonard
3. Kyrie Irving
4. Jimmy Butler 
5. Klay Thompson
6. DeMarcus Cousins
7. Al Horford
8. Kemba Walker
9. Khris Middleton
10. Eric Bledsoe

### Importing Libraries

In [1]:
from bs4 import BeautifulSoup #Web Scraping
import re #Regex
from nltk.tokenize import TweetTokenizer #Tokenizer
import requests #Web Scraping
import urllib3 #Web Scraping
import urllib

### Extracting Wikipedia Links from Google

In [2]:
links1 = []

In [3]:
#Tokenize into 'get' query formats 
query = 'Lebron James'
query = urllib.parse.quote_plus(query)

#Get request to a link; return status code 200 if successful. r.text has all the text saved
r = requests.get('https://www.google.com/search?q=site:wikipedia.com+{}&gbv=1&sei=YwHNVpHLOYiWmQHk3K24Cw'.format(query))

#Convert it to a soup object, but parsing it usin html so it understands the text being parsed. 
soup = BeautifulSoup(r.text, "html.parser")

for item in soup.find_all('h3', attrs={'class' : 'r'}):
    links1.append(item.a['href'][7:])

## Extracting Links from Google Search

In [4]:
links = []

In [5]:
query = "Lebron James Lakers"
google_search = "https://www.google.com/search?sclient=psy-ab&client=ubuntu&hs=k5b&channel=fs&biw=1366&bih=648&noj=1&q=" + query

r = requests.get(google_search)

soup = BeautifulSoup(r.text, "html.parser")

for item in soup.find_all('h3', attrs={'class' : 'r'}):
    links.append(item.a['href'][7:])

In [6]:
#Cleaning the Links
n = len(links)
for i in range(0,n):
    x = links[i].find('&') 
    if x != -1:
        links[i] = links[i][:x]

In [7]:
#Cleaning the links
for i in range(0,n):
    x = links[i].find('%')
    if x != -1:
        links[i] = links[i][:x]

In [13]:
#Get rid of misprint
for i in links:
    if i[0] == '?':
        links.remove(i)

In [14]:
#Displaying the links
n = len(links)
for i in range(0,n):
    print(links[i])

https://www.usatoday.com/story/sports/nba/2018/10/30/lebron-james-lakers-warning-timberwolves/1816624002/
http://www.espn.com/nba/story/_/id/24854936/lebron-james-creates-first-highlights-lakers-preseason-debut
https://www.reuters.com/article/us-basketball-nba-lal-lebron-james/inexperienced-lakers-starting-to-wear-on-lebron-idUSKCN1N424M
https://www.youtube.com/watch
http://www.latimes.com/la-sp-lebron-james-lakers-updates-htmlstory.html
https://lakersstore.com/collections/welcome-lebron-james
https://www.ocregister.com/tag/lebron-james/
https://sports.yahoo.com/lebron-james-teases-lakers-fans-playing-together-kobe-bryant-imagine-213914553.html


In [15]:
#Printing Everything on a page
r = requests.get(links[1])

soup1 = BeautifulSoup(r.content, "html.parser")

text = soup1.find_all('p')

page_text = ""

for item in text:
    str_contents = str(item.contents)
    len_contents = len(str_contents)
    page_text += str_contents[1:len_contents-1]